# Iris from Kaggle to Marvin engine template

## Data Handler

### Acquisitor and Cleanning Action

In [3]:
from marvin_python_toolbox.common.data import MarvinData
import pandas as pd

file_path = MarvinData.download_file(url="https://s3.amazonaws.com/marvin-engines-data/Iris.csv")

iris = pd.read_csv(file_path)
iris.drop('Id',axis=1,inplace=True)

initial_dataset = iris

### Tranning Preparator Action

In [4]:
from sklearn.cross_validation import train_test_split

# Joined dataset 
train, test = train_test_split(initial_dataset, test_size = 0.3)

train_X = train[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
train_y = train.Species

test_X = test[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
test_y =test.Species

# Separeted dataset
petal = initial_dataset[['PetalLengthCm','PetalWidthCm','Species']]
sepal = initial_dataset[['SepalLengthCm','SepalWidthCm','Species']]

train_p, test_p = train_test_split(petal, test_size=0.3, random_state=0)
train_x_p = train_p[['PetalWidthCm','PetalLengthCm']]
train_y_p = train_p.Species
test_x_p = test_p[['PetalWidthCm','PetalLengthCm']]
test_y_p = test_p.Species

train_s, test_s = train_test_split(sepal, test_size=0.3, random_state=0)
train_x_s = train_s[['SepalWidthCm','SepalLengthCm']]
train_y_s = train_s.Species
test_x_s = test_s[['SepalWidthCm','SepalLengthCm']]
test_y_s = test_s.Species

dataset = {
    'petals' : {'train_X': train_x_p, 'train_y': train_y_p, 'test_X': test_x_p, 'test_y': test_y_p},
    'sepals' : {'train_X': train_x_s, 'train_y': train_y_s, 'test_X': test_x_s, 'test_y': test_y_s},
    'joined' : {'train_X': train_X, 'train_y': train_y, 'test_X': test_X, 'test_y': test_y}
}

/home/taka/.virtualenvs/iris-species-engine-env/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Traning Action

In [5]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

algorithms = {
    'svm': svm.SVC, 
    'lr': LogisticRegression,
    'dt': DecisionTreeClassifier,
    'knn': KNeighborsClassifier
}

_model = {}
for name in algorithms.keys():
    algorithm = algorithms[name]
    _model[name + '_petals'] = algorithm().fit(dataset['petals']['train_X'], dataset['petals']['train_y'])
    _model[name + '_sepals'] = algorithm().fit(dataset['sepals']['train_X'], dataset['sepals']['train_y'])
    _model[name + '_joined'] = algorithm().fit(dataset['joined']['train_X'], dataset['joined']['train_y'])
    
model = _model

### Evaluator action

In [6]:
from sklearn import metrics as sk_metrics

_metrics = {}

for m in model.keys():
    dataset_key = m.split("_")[-1]
    
    _test_X = dataset[dataset_key]['test_X']
    _test_y = dataset[dataset_key]['test_y']
    
    model[m].predict(_test_X)
    prediction=model[m].predict(_test_X)
    _metrics[m] = sk_metrics.accuracy_score(prediction, _test_y)

_metrics = sorted(_metrics.iteritems(), key=lambda (k,v): (v,k), reverse=True)

metrics = {
    "best_model" : _metrics[0],
    "all_metrics": _metrics
}

### Prediction Preparator (not used in this example)

### Prediction

In [11]:
input_message = ['12', '34']


final_result = {
    "svm_petals_prediction": model['svm_petals'].predict(input_message)[0],
    "svm_sepals_prediction": model['svm_sepals'].predict(input_message)[0]
}

final_result

/home/taka/.virtualenvs/iris-species-engine-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/taka/.virtualenvs/iris-species-engine-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


{'svm_petals_prediction': 'Iris-virginica',
 'svm_sepals_prediction': 'Iris-virginica'}